In [1]:
import urllib.request
import json
import datetime

OpenStreetMaps API
---
----

#### 1) Coordinates from address with nominatim

https://nominatim.org/release-docs/develop/api/Overview/


In [2]:
def get_coordinates_from_address(address):
    '''
    Returns latitude and longitude from a query, using the nominatim tool
    '''
    formatted_address = address.replace(' ', '+')
    url_request = 'http://nominatim.openstreetmap.org/search?q={}&format=json&polygon=1&addressdetails=1'.format(formatted_address)
    with urllib.request.urlopen(url_request) as response:
        html = response.read()
    location_info = json.loads(html)[0]
    print(location_info['display_name'])
    return float(location_info['lon']), float(location_info['lat'])

start_address = 'Bamako'
dest_address = 'Timbuktu'

print('Start:')
print(get_coordinates_from_address(start_address))
print('---')
print('Destination:')
print(get_coordinates_from_address(dest_address))

Start:
Bamako, Koulikoro, Mali
(-7.984739136241295, 12.61326555)
---
Destination:
Tombouctou, Cercle de Tombouctou, Tombouctou, Mali
(-3.0087272, 16.7719091)


----
#### 2) Fastest Route with OSRM

https://github.com/Project-OSRM/osrm-backend/blob/master/docs/http.md

In [3]:
start_coordinates = 13.388860, 52.517037
end_coordinates = 13.428555, 52.523219

def get_fastest_route_coords(start_coordinates, dest_coordinates, travel_type='driving'):
    '''
    Finds the fastest route between coordinates in the supplied order.
    
    url structure:
    GET /route/v1/{profile}/{coordinates}?alternatives={true|false|number}&steps={true|false}&geometries={polyline|polyline6|geojson}&overview={full|simplified|false}&annotations={true|false}
    '''
    url_request = 'http://router.project-osrm.org/route/v1/{}/{:.6f},{:.6f};{:.6f},{:.6f}?overview=false'.format(
        travel_type, *start_coordinates, *dest_coordinates
    )
    with urllib.request.urlopen(url_request) as response:
        html = response.read()
    routes = json.loads(html)['routes'][0]
    # print(json.loads(html))
    return {'distance': routes['distance'], 'duration': routes['duration']}

get_fastest_route_coords(start_coordinates, end_coordinates)

{'distance': 3795.1, 'duration': 384.4}

---
#### All together

In [4]:
def get_fastest_route(start_address, dest_address):
    '''
    Returns the driving duration and distance between two addresses
    '''
    start_coords = get_coordinates_from_address(start_address)
    dest_coords = get_coordinates_from_address(dest_address)
    
    route = get_fastest_route_coords(start_coords, dest_coords)
    distance = route['distance']/1000
    duration = datetime.timedelta(seconds=route['duration'])
    print('''
          Distance: {:.1f}km
          Time    : {}
          '''.format(distance, duration)
         )
    return {'distance': distance, 'duration': duration}

get_fastest_route('moscou', 'saint-petersbourg')

Москва, Центральный федеральный округ, Россия
Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия

          Distance: 705.1km
          Time    : 7:37:36.300000
          


{'distance': 705.0849000000001,
 'duration': datetime.timedelta(seconds=27456, microseconds=300000)}

Application to csv file
---
----

In [5]:
# Load csv file with start and destination cities
import pandas as pd

itineraries_file_name = 'destinations.csv'
itineraries = pd.read_csv(itineraries_file_name, sep=';')
itineraries = itineraries.loc[:, itineraries.columns[:2]]

In [6]:
def compute_routes_csv(
    itineraries_file_name,
    distance_col='distance (km)',
    duration_col='duration',
    save_inplace=True,
    sep=','):
    '''
    Given a csv file with two columns of start and destination adresses,
    adds columns with the travel time and distance by car.
    
    If `save_inplace` is True (default), modifies the csv inplace, otherwise 
    creates a new csv file.
    '''
    itineraries = pd.read_csv(itineraries_file_name, sep=sep)
    print(itineraries.head())
    # itineraries = itineraries.loc[:, itineraries.columns[:2]]
    distances_durations = [
        get_fastest_route(start, dest) for start, dest in zip(
            itineraries.iloc[:, 0], itineraries.iloc[:, 1])
    ]
    itineraries['duration'] = [d['duration'] for d in distances_durations]
    itineraries['distance (km)'] = [d['distance'] for d in distances_durations]
    
    # Save the results as a csv
    results_file_name = itineraries_file_name if save_inplace else itineraries_file_name + '_travel'
    itineraries.to_csv(itineraries_file_name, index=False)
    print('Saved results to {}'.format(itineraries_file_name))
    
    return itineraries

In [7]:
compute_routes_csv('destinations.csv')

            Départ            Arrivée                   duration   distance  \
0    Saint GAudens   Clermont-Ferrand  0 days 05:06:17.400000000   465.2093   
1  Rio de Janeiro            Brasilia  1 days 12:52:11.200000000  2471.1578   
2           Boston      San Francisco  2 days 07:01:24.600000000  4999.6387   
3           Moscou  Saint Petersbourg  0 days 07:37:36.300000000   705.0849   
4          Beijing           Shanghai  0 days 13:41:54.200000000  1205.8645   

   distance (km)  
0       465.2093  
1      2471.1578  
2      4999.6387  
3       705.0849  
4      1205.8645  
Saint-Gaudens, Haute-Garonne, Occitanie, France métropolitaine, 31800, France
Clermont-Ferrand, Puy-de-Dôme, Auvergne-Rhône-Alpes, France métropolitaine, France

          Distance: 465.2km
          Time    : 5:06:17.400000
          
Rio de Janeiro, Região Geográfica Imediata do Rio de Janeiro, Região Metropolitana do Rio de Janeiro, Região Geográfica Intermediária do Rio de Janeiro, Rio de Janeiro, Região

,Départ,Arrivée,duration,distance,distance (km)
0,Saint GAudens,Clermont-Ferrand,0 days 05:06:17.400000,465.2093,465.2093
1,Rio de Janeiro,Brasilia,1 days 12:52:11.200000,2471.1578,2471.1578
2,Boston,San Francisco,2 days 07:01:24.600000,4999.6387,4999.6387
3,Moscou,Saint Petersbourg,0 days 07:37:36.300000,705.0849,705.0849
4,Beijing,Shanghai,0 days 13:41:54.200000,1205.8645,1205.8645


Searates API
---

In [ ]:
url_request = 'https://tracking.searates.com'
url_request += 'reference?type=BL&number=SII166268&sealine=maeu'
url_request += '&api_key=_your_api_key_'
url_request = 'http://router.project-osrm.org/route/v1/driving/13.388860,52.517037;13.397634,52.529407;13.428555,52.523219?overview=false'
with urllib.request.urlopen(url_request) as response:
    html = response.read()